# Multiverse Knowledge

Make sure the [Multiverse Framework](https://github.com/Multiverse-Framework/Multiverse) is installed properly.

In [1]:
import os

output_dir_path = os.path.join(os.getcwd(), "..", "output")
src_dir_path = os.path.join(os.getcwd(), "..", "src")

import ipywidgets
from IPython.display import display
import zipfile
import subprocess
import solara
import shutil

from utils.display_loading_gif import display_loading_gif

out = ipywidgets.Output()

house_number_widget = ipywidgets.BoundedIntText(
    value=5,
    min=1,
    max=10000,
    step=1,
    description='House nr.:',
    disabled=False
)

def zip_directory(directory_path, output_zip):
    with zipfile.ZipFile(output_zip, "w") as zipf:
        for foldername, subfolders, filenames in os.walk(directory_path):
            for filename in filenames:
                # Create the complete filepath of the file in the directory
                file_path = os.path.join(foldername, filename)
                # Add file to zip
                zipf.write(file_path, os.path.relpath(file_path, directory_path))

@out.capture()
def on_click_get_knowledge(button):
    house_number = house_number_widget.value
    process = subprocess.Popen([f'{src_dir_path}/procthor_to_knowledge.bash', f'{house_number}'])
    loading_widget = display_loading_gif()
    process.wait()

    # Copy the knowledge files to the output directory
    output_house_dir_path = os.path.join(output_dir_path, f"house_{house_number}")
    DUL_owl_path = os.path.join(src_dir_path, "ontology", "DUL.owl")
    USD_owl_path = os.path.join(src_dir_path, "ontology", "USD.owl")
    SOMA_owl_path = os.path.join(src_dir_path, "ontology", "SOMA.owl")
    SOMA_DFL_owl_path = os.path.join(src_dir_path, "ontology", "SOMA_DFL_module.owl")
    shutil.copy(DUL_owl_path, output_house_dir_path)
    shutil.copy(USD_owl_path, output_house_dir_path)
    shutil.copy(SOMA_owl_path, output_house_dir_path)
    shutil.copy(SOMA_DFL_owl_path, output_house_dir_path)

    output_zip_name = f"house_{house_number}_scenes_and_knowledge.zip"
    output_zip_path = os.path.join("/tmp", output_zip_name)
    zip_directory(output_house_dir_path, output_zip_path)

    loading_widget.close()

    display(
        ipywidgets.VBox(
            [
                solara.FileDownload.widget(
                    data=open(output_zip_path, "rb").read(),
                    label=f"Download ZIP (includes USDA, XML, URDF and OWL)",
                    filename=output_zip_name,
                )
            ]
        )
    )

@out.capture()
def on_click_get_house(button):
    house_number = house_number_widget.value
    process = subprocess.Popen([f'{src_dir_path}/procthor_to_scenes.bash', f'{house_number}'])
    loading_widget = display_loading_gif()
    process.wait()
    loading_widget.close()

    output_zip_name = f"house_{house_number}_scenes.zip"
    output_zip_path = os.path.join("/tmp", output_zip_name)
    output_house_dir_path = os.path.join(output_dir_path, f"house_{house_number}")
    zip_directory(output_house_dir_path, output_zip_path)

    get_knowledge_widget = ipywidgets.Button(
        description='Get knowledge',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Get knowledge',
        icon='' # (FontAwesome names without the `fa-` prefix)
    )
    get_knowledge_widget.on_click(on_click_get_knowledge)

    display(
        ipywidgets.VBox(
            [
                solara.FileDownload.widget(
                    data=open(output_zip_path, "rb").read(),
                    label=f"Download ZIP (includes USDA, XML and URDF)",
                    filename=output_zip_name,
                )
            ]
        ),
        get_knowledge_widget
    )

get_house_widget = ipywidgets.Button(
    description='Get house',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Get house',
    icon='' # (FontAwesome names without the `fa-` prefix)
)
get_house_widget.on_click(on_click_get_house)

with out:
    display(house_number_widget, get_house_widget)

out


Output()